<a href="https://colab.research.google.com/github/358Xin/DL/blob/main/Network_Compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --id '1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy' --output food-11.zip

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy
To: /content/food-11.zip
100% 963M/963M [00:06<00:00, 151MB/s]


In [2]:
!unzip -q food-11.zip

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder
from tqdm.auto import tqdm

In [4]:
train_tfm = transforms.Compose([transforms.Resize((142, 142)), transforms.RandomHorizontalFlip(), transforms.RandomRotation(15), transforms.RandomCrop(128), transforms.ToTensor()])
test_tfm = transforms.Compose([transforms.Resize((142, 142)), transforms.CenterCrop(128), transforms.ToTensor()])

In [5]:
batch_size = 64
train_set = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
valid_set = DatasetFolder("food-11/validation", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
unlabeled_set = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
test_set = DatasetFolder("food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [6]:
class StudentNet(nn.Module):
    def __init__(self):
      super(StudentNet, self).__init__()

      self.cnn = nn.Sequential(
        nn.Conv2d(3, 32, 3), 
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Conv2d(32, 32, 3),  
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2, 2, 0),     

        nn.Conv2d(32, 64, 3), 
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2, 2, 0),     

        nn.Conv2d(64, 100, 3), 
        nn.BatchNorm2d(100),
        nn.ReLU(),
        nn.MaxPool2d(2, 2, 0),
        
        # Here we adopt Global Average Pooling for various input size.
        nn.AdaptiveAvgPool2d((1, 1)))
      self.fc = nn.Sequential(nn.Linear(100, 11))
      
    def forward(self, x):
      out = self.cnn(x)
      out = out.view(out.size()[0], -1)
      return self.fc(out)

In [7]:
from torchsummary import summary

student_net = StudentNet()
summary(student_net, (3, 128, 128), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 126, 126]             896
       BatchNorm2d-2         [-1, 32, 126, 126]              64
              ReLU-3         [-1, 32, 126, 126]               0
            Conv2d-4         [-1, 32, 124, 124]           9,248
       BatchNorm2d-5         [-1, 32, 124, 124]              64
              ReLU-6         [-1, 32, 124, 124]               0
         MaxPool2d-7           [-1, 32, 62, 62]               0
            Conv2d-8           [-1, 64, 60, 60]          18,496
       BatchNorm2d-9           [-1, 64, 60, 60]             128
             ReLU-10           [-1, 64, 60, 60]               0
        MaxPool2d-11           [-1, 64, 30, 30]               0
           Conv2d-12          [-1, 100, 28, 28]          57,700
      BatchNorm2d-13          [-1, 100, 28, 28]             200
             ReLU-14          [-1, 100,

In [8]:
def loss_fn_kd(outputs, labels, teacher_outputs, alpha=0.5):
  hard_loss = F.cross_entropy(outputs, labels) * (1 - alpha)
  soft_loss = 0
  return hard_loss + soft_loss

In [23]:
!gdown --id '1B8ljdrxYXJsZv2vmTequdPOofp3VF3NN' --output teacher_resnet18.bin
teacher_net = models.resnet18(pretrained=False, num_classes=11).cuda()
teacher_net.load_state_dict(torch.load(f'./teacher_resnet18.bin'))
teacher_net.eval()

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1B8ljdrxYXJsZv2vmTequdPOofp3VF3NN
To: /content/teacher_resnet18.bin
100% 44.8M/44.8M [00:00<00:00, 239MB/s] 


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [26]:
import enum
device = 'cuda' if torch.cuda.is_available() else 'cpu'
student_net = student_net.to(device)
teacher_net = teacher_net.to(device)

do_semi = True

def get_pseudo_labels(dataset, model):
  loader = DataLoader(dataset, batch_size=batch_size*3, shuffle=False, pin_memory=True)
  pseudo_labels = []
  for batch in tqdm(loader):
    img, _ = batch
    with torch.no_grad():
      logits = model(img.to(device))
      pseudo_labels.append(logits.argmax(dim=-1).detach().cpu())
  pseudo_labels = torch.cat(pseudo_labels)
  for index, ((img,_), pseudo_label) in enumerate(zip(dataset.samples, pseudo_labels)):
    dataset.samples[index] = (img, pseudo_label.item())
  return dataset

if do_semi:
  unlabeled_set = get_pseudo_labels(unlabeled_set, teacher_net)
  concat_dataset = ConcatDataset([train_set, unlabeled_set])
  train_loader = DataLoader(concat_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, drop_last=True)

  0%|          | 0/36 [00:00<?, ?it/s]

In [29]:
from torchvision.transforms.autoaugment import TrivialAugmentWide
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(student_net.parameters(), lr=0.0003, weight_decay=1e-5)
n_epochs = 80

for epoch in range(n_epochs):
  student_net.train()
  
  train_loss = []
  train_acc = []

  for batch in tqdm(train_loader):
    imgs, labels = batch
    logits = student_net(imgs.to(device))
    with torch.no_grad():
      soft_labels = teacher_net(imgs.to(device))
    loss = loss_fn_kd(logits, labels.to(device), soft_labels)
    optimizer.zero_grad()
    loss.backward()
    grad_norm = nn.utils.clip_grad_norm_(student_net.parameters(), max_norm=10)
    optimizer.step()
    acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
    
    train_loss.append(loss.item())
    train_acc.append(acc)
  
  train_loss = sum(train_loss) / len(train_loss)
  train_acc = sum(train_acc) / len(train_acc)
  print(f"[Train | {epoch+1:03d} / {n_epochs:03d}] loss={train_loss:.5f}, acc={train_acc:.5f}")

  student_net.eval()

  valid_loss = []
  valid_acc = []
  for batch in tqdm(valid_loader):
    imgs, labels = batch

    with torch.no_grad():
      logits = student_net(imgs.to(device))
      soft_labels = teacher_net(imgs.to(device))
    loss = loss_fn_kd(logits, labels.to(device), soft_labels)
    acc = (logits.argmax(dim=-1) == labels.to(device)).float().detach().cpu().view(-1).numpy()

    valid_loss.append(loss.item())
    valid_acc += list(acc)

  valid_loss = sum(valid_loss) / len(valid_loss)
  valid_acc = sum(valid_acc) / len(valid_acc)
  print(f"[Valid | {epoch+1:03d} / {n_epochs:03d}] loss={valid_loss:.5f}, acc={valid_acc:.5f}")

  0%|          | 0/154 [00:00<?, ?it/s]

[Train | 001 / 080] loss=0.97769, acc=0.32315


  0%|          | 0/11 [00:00<?, ?it/s]

[Valid | 001 / 080] loss=0.98278, acc=0.30455


  0%|          | 0/154 [00:00<?, ?it/s]

...

[Valid | 079 / 080] loss=0.81653, acc=0.44394


  0%|          | 0/154 [00:00<?, ?it/s]

[Train | 080 / 080] loss=0.66087, acc=0.56037


  0%|          | 0/11 [00:00<?, ?it/s]

[Valid | 080 / 080] loss=0.72973, acc=0.54091


In [30]:
student_net.eval()
predictions = []

for batch in tqdm(test_loader):
  imgs, labels = batch
  with torch.no_grad():
    logits = student_net(imgs.to(device))
  predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

  0%|          | 0/53 [00:00<?, ?it/s]

In [31]:
with open('predict.csv', 'w') as f:
  f.write('Id, Category\n')
  for i, pred in enumerate(predictions):
    f.write(f"{i}, {pred}\n")